# ¿Es determinante el entorno  para los asesinos en serie?
##### Memoria del proyecto

Fuentes de datos y datasets utilizados:

In [2]:
import pandas as pd
df_1 = pd.read_csv("https://www.kaggle.com/datasets/vesuvius13/serial-killers-dataset/data?select=Highest_victim_count.csv") #highest victim count
fuente_1 = "https://www.kaggle.com/datasets/vesuvius13/serial-killers-dataset/data?select=Highest_victim_count.csv"

df_2 = pd.read_json("https://query.data.world/s/xtork6gpqastc7w63cca7pzdkgi5kh?dws=00000") #population density
fuente_2 ="https://data.world/samayo/country-names/workspace/file?filename=country-population-density.json"

df_3 = pd.read_csv('serialkillers_full\\poverty_rate.csv') #poverty rate
fuente_3 = "https://data.oecd.org/inequality/poverty-rate.htm"

df_4 = pd.read_csv("https://www.kaggle.com/datasets/vesuvius13/serial-killers-dataset/data?select=15_to_30_victim_count.csv")
fuente_4 = "https://www.kaggle.com/datasets/vesuvius13/serial-killers-dataset/data?select=15_to_30_victim_count.csv"

df_5 = pd.read_csv("https://gist.githubusercontent.com/radcliff/f09c0f88344a7fcef373/raw/2753c482ad091c54b1822288ad2e4811c021d8ec/wikipedia-iso-country-codes.csv")
fuente_5 = "https://gist.githubusercontent.com/radcliff/f09c0f88344a7fcef373/raw/2753c482ad091c54b1822288ad2e4811c021d8ec/wikipedia-iso-country-codes.csv"

df_6 = pd.read_csv('https://www.kaggle.com/datasets/vesuvius13/serial-killers-dataset/data?select=5_to_14_victim_count.csv')

df_7 = pd.read_csv('https://www.kaggle.com/datasets/vesuvius13/serial-killers-dataset/data?select=Lessthan_5_victim_count.csv')

#CAMBIAR RUTAS a archivos

ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2


In [3]:
# añadir heads

# Limpieza de datos

#### Limpieza de los cuatro datasets de los sujetos
(Este proceso desarrollado se encuentra en el notebook "1datacleaning.ipynb").

+ El primer paso, y el más laborioso, fue el de limpiar los datos.
+ La columna de países de procedencia de los sujetos incluía varios valores en la misma celda (además de datos como "Brasil (supuestamente)"), por lo que he tenido que limpiarla separando los valores en una lista (```str.split()```) y dividiéndolos en filas con ``` pd.explode()```. De esta forma, si un sujeto perpetró crímenes en tres países, aparecerá tres veces, ya que lo que nos importa realmente son los países en los que tienen lugar dichos crímenes.

In [ ]:
hivict['country'].value_counts()

In [ ]:
#separating countries in the same cell as a list
hivict['country']=hivict['country'].str.split('\r\n')

#dividing list in several rows and removing the " (alleged)" bit
hivictexp=hivict.explode('country')
hivictexp['country']=hivictexp['country'].str.replace(" (alleged)","")
hivictexp['country'].unique()

Este proceso se repite para las cuatro tablas de datos de asesinos, que posteriormente se concatenarán.

Conforme iba obteniendo los datos de países limpios, los iba guardando en archivos csv definitivos.

#### Limpieza de los datasets de datos demográficos y la clave de país-código ISO

(Este proceso desarrollado se encuentra en el notebook "1datacleaning.ipynb").

+ Lo primero que tuve que hacer fue homogeinizar los nombres de las columnas y eliminar las que eran irrelevantes:

In [ ]:
#añadir head

In [ ]:
# homogeneizar nombres de columnas
for col in povrates:
    povrates.rename(columns={col:col.lower()}, inplace=True)

povrates.rename(columns={'location':'code', 'time':'year'}, inplace=True)

povrates=povrates[['code', 'year','value']]
povrates.head()

+ Principalmente quería basarme en dos datos demográficos: **la densidad de población y el índice de pobreza**.Sin embargo, el dataset de *poverty rates* incluía el código ISO de los países en lugar de su nombre, lo que me obligó a buscar otro csv que me permitiera hacer la correspondencia entre ambos.


In [ ]:
countrycode=countrycode[['English short name lower case', 'Alpha-3 code']]
countrycode.rename(columns={'English short name lower case':'country name', 'Alpha-3 code': 'code'},inplace=True)
countrycode.sort_values(['code'])

+ Después tuve que limpiar el dataset de índice de pobreza, que tenía varios valores según el año, pero muchos NaN también. **Decidí agruparlos por país y hacer una media del índice de pobreza, para dejar solo una fila por país, que después sería una nueva columna del dataframe.**

In [ ]:
avgpovratecountry=povrates.groupby('code')['value'].mean()
avgpovratecountry=pd.DataFrame(avgpovratecountry).reset_index()
avgpovratecountry.rename(columns={'value':'mean value'},inplace=True)
avgpovratecountry.drop(columns=['level_0','index'],inplace=True)

+ Una vez hecho esto, se podía eliminar la columna de año y los valores nulos.

In [ ]:
povrates.dropna(subset=['value'],inplace=True)
povrates.dropna(subset=['year'],inplace=True)
simplepovrates=povrates.drop(columns='year').drop_duplicates('code')

+ A partir de ahí, tuve que empezar a utilizar ```pd.merge()``` para juntar los datos demográficos y la clave de país-código ISO.

In [ ]:
cleanpovrate=simplepovrates.merge(avgpovratecountry, on='code').drop(columns='value')

+ Se añade el nombre de los países a *pov rates* haciendo un merge con la anterior tabla.

In [ ]:
countrypovrate=pd.merge(cleanpovrate, countrycode, how='outer')
countrypovrate.dropna(subset=['mean value'],inplace=True)

#### Homogeinizar los nombres de los países en los distintos datasets


+ **Ahí me di cuenta de que algunos países del dataset de asesinos no estaban escritos igual que los de este dataset** (por ejemplo, 'Korea, Republic of', frente a 'South Korea').
+ Esto me obligó a crear un diccionario para corregir y homogeninizar los nombres de los países:

In [ ]:
wrongpovratecountries={'Korea, Republic of':'South Korea'}

for wrong,right in wrongpovratecountries.items():
    countrypovrate['country name']=countrypovrate['country name'].str.replace(wrong, right)

+ El mismo procedimiento se repitió para el dataest de la densidad de población:

In [ ]:
wrongpopdenscountries={'Iran, Islamic Rep.':'Iran','Russian Federation':'Russia','Korea, Rep.':'South Korea',
                       'Venezuela, RB':'Venezuela','Yemen, Rep.':'Yemen'}

for wrong,right in wrongpovratecountries.items():
    popdens['country']=popdens['country'].str.replace(wrong, right)

(Este proceso desarrollado se encuentra en el notebook "2datacleaning_matching.ipynb").

+ Se concatenan los cuatro datasets de datos de asesinos con ```pd.concat()``` y se ve los países cuyo nombre no encaja con los otros datasets (para hacer otro diccionario).

In [ ]:
#keys are wrong, values are the correct ones (in killer df)
wrongkillercountries={'Allied-occupied Germany':'Germany','Austria-Hungary':'Hungary',
                      'Czechoslovakia':'Czech Republic','East Germany':'Germany','German Empire':'Germany',
                      'Kingdom of Romania':'Romania','Ottoman Empire':'Iran','Portuguese Angola':'Angola',
                      'West Germany':'Germany', 'Soviet Union':'Ukraine', 'Yugoslavia':'Serbia'}

for wrong,right in wrongkillercountries.items():
    totalvict['country']=totalvict['country'].str.replace(wrong, right)

(Este proceso desarrollado se encuentra en el notebook "3mergingdf.ipynb").

+ Por último, en un último Jupyter notebook de limpieza se fusionan todos los datasets para poder trabajar con un dataframe definitivo y se eliminan las columnas innecesarias.

In [ ]:
#merged killers df with poverty rate df
kill_povrate=killers.merge(povrate, how='left')

kill_povrate['mean value'].isnull().value_counts() #quite a few nans for poverty rate

#remove "code" column
kill_povrate.drop(columns='code', inplace=True)

In [ ]:
#merged killers_povrate df with pop density df
dfkillers=kill_povrate.merge(popdens, how='left')
dfkillers.rename(columns={'mean value':'pov rate'},inplace=True)

dfkillers.to_csv("..\\data\\cleaned\\eda_df.csv")

+ El *eda_df.csv* ya está listo para el análisis. Veamos el resultado final:

In [ ]:
len(dfkillers[(dfkillers['density'].notnull()) & (dfkillers['pov rate'].notnull())])
#201 killers with both data
#pov rate: 205 values, 150 nan
#density: 313 values, 42 nan

+ Hay 150 países sin datos de índice de pobreza y 42 países sin datos de densidad de población. **Es decir, disponemos de todos los datos de 201 sujetos**.
+ Se exporta otro csv sin NaN también:

In [ ]:
notnullkillers=dfkillers[(dfkillers['density'].notnull()) & (dfkillers['pov rate'].notnull())]
notnullkillers.to_csv("..\\data\\cleaned\\notnull_df.csv")

# Análisis de los datos y comprobación de hipótesis